In [1]:
import itertools

from extract import extract_dataset_archive
from split import convert_train_set_to_images
import os
from tensorflow.keras.layers import Dense, LSTM, Activation, Flatten, TimeDistributed, ConvLSTM2D, Conv3D, BatchNormalization
from tensorflow.keras.models import Sequential
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import MinMaxScaler 
from sklearn.preprocessing import minmax_scale
# from tensorflow.keras.layers.convolutional import 
# from tensorflow.keras.layers.convolutional_recurrent import ConvLSTM2D
# from tensorflow.keras.layers.normalization import BatchNormalization
# os.system('python scripts/demo_inference.py --cfg pretrained_models/256x192_res152_lr1e-3_1x-duc.yaml --checkpoint  pretrained_models/fast_421_res152_256x192.pth --video ../dataset/NonViolence/NV_1.mp4 --outdir ../dataset --sp')

def prepare_dataset():
    extract_dataset_archive()
    convert_train_set_to_images()


def convert_videos_to_json(path_in, path_out):
    os.system('python scripts/demo_inference.py --cfg pretrained_models/256x192_res152_lr1e-3_1x-duc.yaml '
              '--checkpoint  pretrained_models/fast_421_res152_256x192.pth --video ' + path_in + ' --outdir ' + path_out + ' --sp')


#prepare_dataset()
#os.chdir("AlphaPose")

# for i in range(1, 1001):
#    file_in = str("../dataset/Violence/V_") + str(i) + ".mp4"
#    file_out = str("../dataset/results/V_") + str(i) + "_json"
#    convert_videos_to_json(file_in, file_out)

#for i in range(1, 1001):
#    file_in = str("../dataset/NonViolence/NV_") + str(i) + ".mp4"
#    file_out = str("../dataset/results/NV_") + str(i) + "_json"
#    convert_videos_to_json(file_in, file_out)



person_count = 5
max_frames = 280


C:\Users\Jack\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Jack\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Jack\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Jack\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (

In [2]:
model = Sequential()
model.add(TimeDistributed(Flatten(input_shape=(5, 68))))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.build(input_shape=(280, 5, 68))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri multiple                  0         
_________________________________________________________________
lstm (LSTM)                  multiple                  67600     
_________________________________________________________________
dense (Dense)                multiple                  101       
Total params: 67,701
Trainable params: 67,701
Non-trainable params: 0
_________________________________________________________________


In [3]:
csv_dir = "dataset/csvs"

csv_files = os.listdir(csv_dir)

train_files, test_files = train_test_split(csv_files, train_size=0.8)

In [5]:
x_train = []
y_train = []

trained = 0

x_test = []
y_test = []

tested = 0

In [6]:
for file_name in train_files:
    
    if trained == 100:
        break
    trained += 1
    
    csv = pd.read_csv(csv_dir + "/" + file_name)
    
    current_frame = 0
    
    violent = 0
    print(file_name)
    if len(csv) > 0:
        violent = csv.violent[0]

        csv = csv.drop('violent',axis=1)

        labels = csv.columns.values

        for label in labels[2:]:
            minmax_scale = preprocessing.MinMaxScaler().fit(csv[[label]])
            csv[[label]] = minmax_scale.transform(csv[[label]])

        data = csv.to_numpy()

        val_cumulative = {}
        frame_ids = set()
        for person in data:
            frame_ids.add(person[0])
            frame, person_id = person[:2]

            vel_array = []
            for vel in range(5, 70, 4):
                vel_array.append(vel)

            total_vel = sum(vel_array)

            if person_id in val_cumulative:
                val_cumulative[person_id] += total_vel
            else:
                val_cumulative[person_id] = 0


        val_cumulative = {k: v for k, v in sorted(val_cumulative.items(), key=lambda item: item[1], reverse=True)}
        top_people = dict(itertools.islice(val_cumulative.items(), person_count))
        top_people_dataset = {}

        starter_map = {}
        for key in top_people.keys():
            starter_map[key] = np.zeros(68)

        for frame_id in frame_ids:
            # Trim to 280 if above
            if current_frame == max_frames:
                break
            tmp_map = starter_map
            people = np.array([item for item in data if item[0] == frame_id and item[1] in top_people.keys()])
            for person in people:
                tmp_map[person[1]] = person[2:]
                
            padded = list(tmp_map.values())
                
            for i in range(len(padded), person_count):
                padded.append(np.zeros(68))

            top_people_dataset[frame_id] = np.asarray(padded)
            current_frame += 1
            
    
    # Pad to 280 frames if below
    for i in range(current_frame+1, max_frames+1):
        top_people_dataset[i] = []
        for j in range(0, person_count):
            top_people_dataset[i].append(np.zeros(68))
                 
    tmp = np.asarray(list(top_people_dataset.values()))
    
    print(tmp.shape)
    
    for entry in tmp:
        x_train.append(entry)
    
    if violent == 0:
        for entry in np.zeros(len(tmp)):
            y_train.append(entry)
    else:
        for entry in np.ones(len(tmp)):
            y_train.append(entry)
    

NV_477_json.csv
(280, 5, 68)
V_421_json.csv
(280, 5, 68)
V_967_json.csv
(280, 5, 68)
NV_99_json.csv
(280, 5, 68)
NV_800_json.csv
(280, 5, 68)
V_378_json.csv
(280, 5, 68)
V_222_json.csv
(280, 5, 68)
NV_306_json.csv
(280, 5, 68)
NV_760_json.csv
(280, 5, 68)
NV_172_json.csv
(280, 5, 68)
V_743_json.csv
(280, 5, 68)
V_157_json.csv
(280, 5, 68)
NV_834_json.csv
(280, 5, 68)
NV_687_json.csv
(280, 5, 68)
V_931_json.csv
(280, 5, 68)
V_528_json.csv
(280, 5, 68)
V_448_json.csv
(280, 5, 68)
NV_211_json.csv
(280, 5, 68)
V_89_json.csv
(280, 5, 68)
V_547_json.csv
(280, 5, 68)
NV_743_json.csv
(280, 5, 68)
NV_722_json.csv
(280, 5, 68)
V_583_json.csv
(280, 5, 68)
V_450_json.csv
(280, 5, 68)
V_264_json.csv
(280, 5, 68)
V_472_json.csv
(280, 5, 68)
V_870_json.csv
(280, 5, 68)
NV_156_json.csv
(280, 5, 68)
NV_177_json.csv
(280, 5, 68)
NV_17_json.csv
(280, 5, 68)
NV_840_json.csv
(280, 5, 68)
NV_731_json.csv
(280, 5, 68)
NV_383_json.csv
(280, 5, 68)
NV_129_json.csv
(280, 5, 68)
V_148_json.csv
(280, 5, 68)
V_535

In [7]:
x_train = np.asarray(list(x_train))
y_train = np.asarray(list(y_train))
model.fit(x_train, y_train, epochs=20, batch_size=100, validation_split=0.2)
scores = model.evaluate(x_train, y_train, verbose=1)

Train on 22400 samples, validate on 5600 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
22400/22400 [==============================] - 3s 133us/sample - loss: 0.6066 - acc: 0.6297 - val_loss: 0.6521 - val_acc: 0.6288
Epoch 2/20
22400/22400 [==============================] - 2s 94us/sample - loss: 0.5282 - acc: 0.6781 - val_loss: 0.7366 - val_acc: 0.6096
Epoch 3/20
22400/22400 [==============================] - 2s 88us/sample - loss: 0.4823 - acc: 0.6993 - val_loss: 0.7541 - val_acc: 0.6127
Epoch 4/20
22400/22400 [==============================] - 2s 90us/sample - loss: 0.4583 - acc: 0.7104 - val_loss: 0.7844 - val_acc: 0.6141
Epoch 5/20
22400/22400 [==============================] - 2s 97us/sample - loss: 0.4365 - acc: 0.7143 - val_loss: 0.7906 - val_acc: 0.6273
Epoch 6/20
22400/22400 [==============================] - 2s 96us/sample - loss: 0.4262 - acc: 0.7213 - val_loss: 0.7568 - val_acc: 0.6330
Epoch 7/20
22400/22400 [==============================] - 2s 89us/sa

In [8]:
for file_name in test_files:
    csv = pd.read_csv(csv_dir + "/" + file_name)
    
    if tested == 100:
        break
    tested += 1
    
    violent = csv.violent[0]
    
    # Resize all CSV files to same arbitrary limit
    csv = csv.drop('violent',axis=1)
    
    labels = csv.columns.values
    
    
    for label in labels[2:]:
        minmax_scale = preprocessing.MinMaxScaler().fit(csv[[label]])
        csv[[label]] = minmax_scale.transform(csv[[label]])
    
    data = csv.to_numpy()

    val_cumulative = {}
    frame_ids = set()
    for person in data:
        frame_ids.add(person[0])
        frame, person_id = person[:2]

        vel_array = []
        for vel in range(5, 70, 4):
            vel_array.append(vel)

        total_vel = sum(vel_array)

        if person_id in val_cumulative:
            val_cumulative[person_id] += total_vel
        else:
            val_cumulative[person_id] = 0
        
        
    val_cumulative = {k: v for k, v in sorted(val_cumulative.items(), key=lambda item: item[1], reverse=True)}
    top_people = dict(itertools.islice(val_cumulative.items(), person_count))
    top_people_dataset = {}
    
    current_frame = 0
    
    starter_map = {}
    for key in top_people.keys():
        starter_map[key] = np.zeros(68)

    for frame_id in frame_ids:
        # Trim to 280 if above
        if current_frame == max_frames:
            break
        tmp_map = starter_map
        people = np.array([item for item in data if item[0] == frame_id and item[1] in top_people.keys()])
        for person in people:
            tmp_map[person[1]] = person[2:]
                
        padded = list(tmp_map.values())
                
        for i in range(len(padded), person_count):
            padded.append(np.zeros(68))

        top_people_dataset[frame_id] = np.asarray(padded)
        current_frame += 1
            
    
    # Pad to 280 frames if below
    for i in range(current_frame+1, max_frames+1):
        top_people_dataset[i] = []
        for j in range(0, person_count):
            top_people_dataset[i].append(np.zeros(68))
                 
                
    tmp = np.asarray(list(top_people_dataset.values()))
    
    for entry in tmp:
        x_test.append(entry)
    
    if violent == 0:
        for entry in np.zeros(len(tmp)):
            y_test.append(entry)
    else:
        for entry in np.ones(len(tmp)):
            y_test.append(entry)

In [9]:
x_test = np.asarray(list(x_test))
y_test = np.asarray(list(y_test))

# model.fit(x_train, y_train, epochs=10, batch_size=100, validation_split=0.2)
scores = model.evaluate(x_test, y_test, verbose=1)
scores

28000/28000 [==============================] - 2s 64us/sample - loss: 1.1851 - acc: 0.5669


[1.1850561106053166, 0.56685716]

In [ ]:
output.shape